In [1]:
# Import libaries

from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np

In [2]:
final_list = []

# Skytrax URL
url = 'https://www.airlinequality.com/airline-reviews/pia-pakistan-international-airlines/?sortby=post_date%3ADesc&pagesize=100'

# Request URL
response = requests.get(url)

# Create Soup
soup = BeautifulSoup(response.content, 'lxml')

# For loop for HTML
for idx, review in enumerate(soup.find_all('div',{'class':"review-stats"})):
    
    # Create a empty dictionary to contain columns and values
    review_dict = dict()
    
    # Values that is in star fill
    names = review.find_all('td', class_='review-rating-header')[::-1]
    ratings = review.find_all('td', class_='stars')[::-1]
    
    try:
        for name, rating in zip(names[1:], ratings):
            review_dict[name.text] = rating.find_all(class_='fill')[-1].text
    except:
        review_dict[name.text] = np.nan
    
    # Author, Date, Overall_rating,Text_content, Recommended
    try:
        review_dict['Author'] = soup.find_all(itemprop='name')[idx+1].text
        review_dict['Date'] = soup.find_all(itemprop='datePublished')[::2][idx].attrs['content']
        review_dict['text_content'] = soup.find_all('div',class_='text_content')[idx].text
        review_dict['Recommended'] = soup.find_all('div',{'class':"review-stats"})[idx].find_all('tr')[-1].find_all('td')[-1].text
    
    except:
        review_dict['Overall_rating'] = np.nan
        
    # Type of Traveller
    try:
        review.find('td',{'class':'review-rating-header type_of_traveller'}).text is not None
        review_dict['traveller_type'] = soup.find_all('td',{'class':'review-rating-header type_of_traveller'})[idx].fetchNextSiblings()[0].text
    
    except:
        review_dict['traveller_type'] = np.nan
    
    # Seat type
    try:
        review.find('td',{'class':'review-rating-header cabin_flown'}).text is not None
        review_dict['seat_type'] = soup.find_all('td',{'class':'review-rating-header cabin_flown'})[idx].fetchNextSiblings()[0].text
    
    except:
        review_dict['seat_type'] = np.nan
    
    # Overall Rating
    try:
        review_dict['Overall_rating'] = soup.find_all('div',{'itemprop':'reviewRating'})[idx].find(itemprop="ratingValue").text
    
    except:
        review_dict['Overall_rating'] = np.nan
    
    # Append dictionary into a list
    final_list.append(review_dict)

# Create DataFrame
df = pd.DataFrame(final_list)
df['airline'] = 'pakistan intl airlines'

# rearrange dataframe
df = df[['airline','Author','Date','text_content','traveller_type','seat_type','Overall_rating','Seat Comfort',
 'Cabin Staff Service','Food & Beverages','Inflight Entertainment','Ground Service','Value For Money',  
  'Recommended']]

# rename dataframe
df.columns = ['airline_name', 'author', 'date', 'content', 'type_traveller','cabin_flown', 'overall_rating', 'seat_comfort_rating',
'cabin_staff_rating', 'food_beverages_rating','inflight_entertainment_rating', 'ground_service_rating','value_money_rating', 'recommended']

# Save to CSV
df.to_csv('Data/pakistan.csv',index=False)

In [5]:
# Sanity check
df.tail()

,airline_name,author,date,content,type_traveller,cabin_flown,overall_rating,seat_comfort_rating,cabin_staff_rating,food_beverages_rating,inflight_entertainment_rating,ground_service_rating,value_money_rating,recommended
95,pakistan intl airlines,Huzayfah Alam,2013-12-11,We were flying from Lahore Punjab Pakistan to ...,NaN,Economy Class,NaN,4,5,4,3,NaN,3,yes
96,pakistan intl airlines,R Edward,2013-12-10,Travelled PIA for the first time (LHR-ISB PK78...,NaN,Economy Class,NaN,4,5,5,3,NaN,4,yes
97,pakistan intl airlines,R Iqbal,2013-12-09,London Heathrow to Lahore Pakistan PK758 2 Nov...,NaN,Economy Class,NaN,3,4,1,1,NaN,2,no
98,pakistan intl airlines,Altaf Ayesha,2013-12-04,Been using PIA for several years. Mainly becau...,NaN,Economy Class,NaN,4,5,5,3,NaN,3,no
99,pakistan intl airlines,Mohammad Asif,2013-10-20,Flew from Multan-Jeddah on PK-731. Basically i...,NaN,Premium Economy,NaN,4,5,5,2,NaN,3,yes


In [4]:
df.head()

,airline_name,author,date,content,type_traveller,cabin_flown,overall_rating,seat_comfort_rating,cabin_staff_rating,food_beverages_rating,inflight_entertainment_rating,ground_service_rating,value_money_rating,recommended
0,pakistan intl airlines,A Sarqir,2020-06-04,✅ Trip Verified | I will never use this airli...,Family Leisure,Economy Class,1,1,1,1,NaN,1,1,no
1,pakistan intl airlines,L Malban,2020-03-27,✅ Trip Verified | Birmingham to Islamabad. I ...,Solo Leisure,Economy Class,1,NaN,NaN,NaN,NaN,NaN,1,no
2,pakistan intl airlines,A Khan,2020-03-14,✅ Trip Verified | Flew executive economy from...,Solo Leisure,Premium Economy,3,2,3,3,1,3,2,no
3,pakistan intl airlines,J Khan,2020-02-24,Not Verified | Manchester to Lahore. I went w...,Family Leisure,Economy Class,1,1,1,3,NaN,1,1,no
4,pakistan intl airlines,Niknam Hussain,2020-02-08,✅ Trip Verified | Birmingham to Islamabad. Tr...,Family Leisure,Premium Economy,3,3,5,3,NaN,1,1,no
